In [79]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import os 
import time

## 讀入資料

In [80]:
# Read the training data
jade_train = pd.read_csv('train.zip', compression='zip', 
                         header=0, sep=',', quotechar='"')

In [81]:
# Read the testing data
jade_test = pd.read_csv('test.zip', compression='zip', 
                         header=0, sep=',', quotechar='"')

In [82]:
#The descriptions of variables
variable_meanings = pd.read_csv("variable_meanings.csv")
variable_meanings

,欄位,說明
0,acqic,收單行代碼
1,bacno,歸戶帳號
2,cano,交易卡號
3,conam,交易金額-台幣（經過轉換）
4,contp,交易類別
5,csmcu,消費地幣別
6,ecfg,網路交易註記
7,etymd,交易型態
8,flbmk,Fallback 註記
9,flg_3dsmk,3DS 交易註記


In [83]:
jade_train.head(20)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,113261,38038,513.80,5,0,N,0,N,N,...,0,33,172652.0,457,59333,N,0,102,0,516056
1,0,134508,45725,465.62,5,0,N,2,N,N,...,0,9,105114.0,451,0,N,5817,102,0,4376
2,6881,15408,188328,513.80,5,0,N,0,N,N,...,0,6,152458.0,457,59333,N,0,102,0,483434
3,6716,157159,29967,1016.11,5,62,N,5,N,N,...,0,5,172946.0,247,50436,N,3281,102,0,1407164
4,5975,105985,81305,713.66,5,62,N,4,N,N,...,0,6,182129.0,263,93775,N,5817,102,0,1051004
5,0,78377,2295,465.62,5,0,N,2,N,N,...,0,6,104918.0,451,0,N,5817,102,0,2943
6,6411,94435,49219,1806.49,3,62,N,4,N,N,...,0,6,172624.0,339,0,N,5865,102,0,1622153
7,6769,112032,177989,526.88,6,62,N,2,N,N,...,0,7,34933.0,373,79200,N,5817,102,0,57795
8,6092,92294,85535,201.39,2,62,N,2,N,N,...,0,6,64652.0,264,8335,N,3585,102,0,836165
9,0,16279,110755,465.62,5,0,N,2,N,N,...,0,10,104447.0,451,0,N,5817,102,0,4507


In [84]:
jade_train.describe()

,acqic,bacno,cano,conam,contp,csmcu,etymd,fraud_ind,hcefg,iterm,locdt,loctm,mcc,mchno,scity,stocn,stscd,txkey
count,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06
mean,6.008003e+03,8.209027e+04,1.089170e+05,6.547219e+02,4.829368e+00,5.383324e+01,4.149114e+00,1.337572e-02,4.749410e+00,4.962784e-02,4.532732e+01,1.463152e+05,2.978089e+02,5.589022e+04,4.755128e+03,9.565116e+01,2.485499e-02,9.711265e+05
std,1.502420e+03,4.736249e+04,6.090363e+04,4.028078e+02,6.513408e-01,2.072135e+01,2.394259e+00,1.148774e-01,1.111927e+00,3.652493e-01,2.601889e+01,5.212107e+04,7.796778e+01,3.082297e+04,1.979815e+03,1.890027e+01,2.216804e-01,5.641322e+05
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.982000e+03,4.140300e+04,5.763500e+04,4.370600e+02,5.000000e+00,6.200000e+01,2.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00,2.300000e+01,1.109310e+05,2.500000e+02,3.377400e+04,3.795000e+03,1.020000e+02,0.000000e+00,4.869475e+05
50%,6.716000e+03,8.191200e+04,1.097850e+05,5.919000e+02,5.000000e+00,6.200000e+01,4.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00,4.500000e+01,1.515220e+05,2.640000e+02,5.936000e+04,5.817000e+03,1.020000e+02,0.000000e+00,9.795800e+05
75%,6.769000e+03,1.230590e+05,1.618760e+05,8.073400e+02,5.000000e+00,6.200000e+01,5.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00,6.800000e+01,1.858270e+05,3.430000e+02,7.920000e+04,5.817000e+03,1.020000e+02,0.000000e+00,1.455200e+06
max,6.884000e+03,1.638840e+05,2.133340e+05,7.208770e+03,6.000000e+00,7.500000e+01,1.000000e+01,1.000000e+00,9.000000e+00,8.000000e+00,9.000000e+01,2.359590e+05,4.590000e+02,1.033070e+05,6.671000e+03,1.070000e+02,4.000000e+00,1.958239e+06


In [85]:
jade_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1521787 entries, 0 to 1521786
Data columns (total 23 columns):
acqic        1521787 non-null int64
bacno        1521787 non-null int64
cano         1521787 non-null int64
conam        1521787 non-null float64
contp        1521787 non-null int64
csmcu        1521787 non-null int64
ecfg         1521787 non-null object
etymd        1521787 non-null int64
flbmk        1509206 non-null object
flg_3dsmk    1509206 non-null object
fraud_ind    1521787 non-null int64
hcefg        1521787 non-null int64
insfg        1521787 non-null object
iterm        1521787 non-null int64
locdt        1521787 non-null int64
loctm        1521787 non-null float64
mcc          1521787 non-null int64
mchno        1521787 non-null int64
ovrlt        1521787 non-null object
scity        1521787 non-null int64
stocn        1521787 non-null int64
stscd        1521787 non-null int64
txkey        1521787 non-null int64
dtypes: float64(2), int64(16), object(5)
memory us

- We can know that "flbmk", "flg_3dsmk" have missing value
- **Nominal variables**: bacno, cano, txkey	

- **Categorical variables**: acqic, contp, csmcu, ecfg, etymd, flbmk, flg_3dsmk, fraud_ind, hcefg, insfg, mcc, mchno, ovrlt, scity, stocn, stscd

In [86]:
# the shape of the training and testing data
print(jade_train.shape) #(1521787, 23)
print(jade_test.shape) # (421665, 22)

(1521787, 23)
(421665, 22)


## Data Preprocessing

### Extract hour, minute, second from variable "loctm"（授權時間）

In [87]:
jade_train["hour"]=jade_train["loctm"]//10000
jade_train["minute"]=jade_train["loctm"]//100 - (jade_train["loctm"]//10000)*100
jade_train["second"]=jade_train["loctm"]%100
jade_train = jade_train.drop("loctm",axis=1)
jade_train.head()

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,hour,minute,second
0,6881,113261,38038,513.80,5,0,N,0,N,N,...,457,59333,N,0,102,0,516056,17.0,26.0,52.0
1,0,134508,45725,465.62,5,0,N,2,N,N,...,451,0,N,5817,102,0,4376,10.0,51.0,14.0
2,6881,15408,188328,513.80,5,0,N,0,N,N,...,457,59333,N,0,102,0,483434,15.0,24.0,58.0
3,6716,157159,29967,1016.11,5,62,N,5,N,N,...,247,50436,N,3281,102,0,1407164,17.0,29.0,46.0
4,5975,105985,81305,713.66,5,62,N,4,N,N,...,263,93775,N,5817,102,0,1051004,18.0,21.0,29.0


In [88]:
jade_test["hour"]=jade_test["loctm"]//10000
jade_test["minute"]=jade_test["loctm"]//100 - (jade_test["loctm"]//10000)*100
jade_test["second"]=jade_test["loctm"]%100
jade_test = jade_test.drop("loctm",axis=1)
jade_test.head()

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,hour,minute,second
0,6881,163188,116799,513.8,5,0,N,0,N,N,...,457,59360,N,0,102,0,592489,21.0,53.0,28.0
1,6881,163188,116799,513.8,5,0,N,0,N,N,...,457,59360,N,0,102,0,592452,22.0,20.0,7.0
2,6881,163188,116799,513.8,5,0,N,0,N,N,...,457,59360,N,0,102,0,590212,17.0,0.0,13.0
3,6881,163188,116799,513.8,5,0,N,0,N,N,...,457,59360,N,0,102,0,590209,16.0,59.0,14.0
4,6881,163188,116799,513.8,5,0,N,0,N,N,...,457,59360,N,0,102,0,592488,21.0,53.0,11.0


In [89]:
print(jade_train.shape)# 23 columns became 25 columns
print(jade_test.shape)# 22 columns became 24 columns

(1521787, 25)
(421665, 24)


In [90]:
# Used forward-fill to impute jade_train's missing value
jade_train.fillna(method='ffill',inplace=True)

In [91]:
# Used forward-fill to impute jade_test's missing value
jade_test.fillna(method='ffill',inplace=True)

In [71]:
# #把數字很大筆的類別資料先移除不看（收單行代碼、歸戶帳號、交易卡號、特店代號、交易序號）
# jade_train = jade_train.drop(columns = ['acqic','bacno','cano','mchno','txkey'])
# print(jade_train.shape) # 23 columns became 20 columns

(1521787, 20)


In [72]:
# def one_hot_encoder(combined):
#     categorical_variables = ["ecfg","insfg","ovrlt","flbmk","flg_3dsmk"]
#     one_hot_encoded = pd.get_dummies(combined,columns=categorical_variables, dummy_na = True)
#     print("============[Data types]==============")
#     print(one_hot_encoded.dtypes)
#     return one_hot_encoded

# jade_train_combined = one_hot_encoder(jade_train)

============[Data types]==============
conam            float64
contp              int64
csmcu              int64
etymd              int64
fraud_ind          int64
hcefg              int64
iterm              int64
locdt              int64
mcc                int64
scity              int64
stocn              int64
stscd              int64
hour             float64
minute           float64
second           float64
ecfg_N             uint8
ecfg_Y             uint8
ecfg_nan           uint8
insfg_N            uint8
insfg_Y            uint8
insfg_nan          uint8
ovrlt_N            uint8
ovrlt_Y            uint8
ovrlt_nan          uint8
flbmk_N            uint8
flbmk_Y            uint8
flbmk_nan          uint8
flg_3dsmk_N        uint8
flg_3dsmk_Y        uint8
flg_3dsmk_nan      uint8
dtype: object


In [92]:
#把 jade_train 做 Y/N 轉換
jade_train.ecfg = jade_train.ecfg.map(dict(Y=1, N=0))
jade_train.flbmk = jade_train.flbmk.map(dict(Y=1, N=0))
jade_train.flg_3dsmk = jade_train.flg_3dsmk.map(dict(Y=1, N=0))
jade_train.insfg = jade_train.insfg.map(dict(Y=1, N=0))
jade_train.ovrlt = jade_train.ovrlt.map(dict(Y=1, N=0))

In [93]:
#把 jade_test 做 Y/N 轉換
jade_test.ecfg = jade_test.ecfg.map(dict(Y=1, N=0))
jade_test.flbmk = jade_test.flbmk.map(dict(Y=1, N=0))
jade_test.flg_3dsmk = jade_test.flg_3dsmk.map(dict(Y=1, N=0))
jade_test.insfg = jade_test.insfg.map(dict(Y=1, N=0))
jade_test.ovrlt = jade_test.ovrlt.map(dict(Y=1, N=0))

In [94]:
#檢測是否還存在NAN
print(np.where(np.isnan(jade_train)))
print(np.where(np.isnan(jade_test)))

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [95]:
print(jade_train.shape)
print(jade_test.shape)

(1521787, 25)
(421665, 24)


In [96]:
#把 jade_train 分 70% train, 30% test
train_X, test_X, train_y, test_y = train_test_split(jade_train.drop(columns = ['fraud_ind']), 
                                                    jade_train['fraud_ind'],
                                                    train_size = 0.7,
                                                    random_state = 123
                                                    #stratify = jade_train['fraud_ind']
                                                   )


In [97]:
# 不切 70%, 30% 的切法
# train_X = pd.DataFrame(jade_train.drop(columns = ['fraud_ind']))
# train_y = jade_train['fraud_ind']


In [98]:
print(train_X.shape,
test_X.shape,
train_y.shape,
test_y.shape)

(1065250, 24) (456537, 24) (1065250,) (456537,)


# Random Forest

In [99]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import ensemble, preprocessing
np.random.seed(0)

In [100]:
# 建立 random forest 模型
forest = ensemble.RandomForestClassifier(n_estimators = 100)
forest_fit = forest.fit(train_X, train_y)

test_y_predicted = forest.predict(test_X)

In [101]:
# 績效
accuracy = metrics.accuracy_score(test_y, test_y_predicted)
print(accuracy)

print(forest.score(test_X, test_y))

0.9933652694086131
0.9933652694086131


因為盜刷的比例佔所有資料的比例很低，就算全部都猜沒有盜刷，那麼 accuracy 也還是會很高，所以 accuracy 不太有參考意義

In [102]:
#把玉山的 test data 放入 model 做預測
test_y_predicted_jade_test = forest.predict(jade_test)

print(test_y_predicted_jade_test)
test_y_predicted_jade_test.shape

[0 0 0 ... 0 0 0]


(421665,)

### 檢視 feature importances 

In [103]:
forest.feature_importances_

array([0.04347174, 0.08444421, 0.09271396, 0.11561334, 0.00257416,
       0.04409357, 0.01194269, 0.01099631, 0.00069117, 0.00150971,
       0.00230716, 0.00012072, 0.00017483, 0.06574504, 0.04016558,
       0.05009837, 0.00406143, 0.06513424, 0.0561198 , 0.06124717,
       0.09061096, 0.05218672, 0.05429386, 0.04968323])

In [104]:
import pandas as pd
feature_importances = pd.DataFrame(forest.feature_importances_,
                                   index = train_X.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

           importance
conam        0.115613
cano         0.092714
txkey        0.090611
bacno        0.084444
locdt        0.065745
scity        0.065134
stscd        0.061247
stocn        0.056120
minute       0.054294
hour         0.052187
mchno        0.050098
second       0.049683
csmcu        0.044094
acqic        0.043472
mcc          0.040166
ecfg         0.011943
etymd        0.010996
ovrlt        0.004061
contp        0.002574
hcefg        0.002307
flg_3dsmk    0.001510
flbmk        0.000691
iterm        0.000175
insfg        0.000121


In [105]:
#取出前10個 features
print(feature_importances.index[:10])

Index(['conam', 'cano', 'txkey', 'bacno', 'locdt', 'scity', 'stscd', 'stocn',
       'minute', 'hour'],
      dtype='object')


In [106]:
#用前10個 features 再建一個新的 model
forest_10 = ensemble.RandomForestClassifier(n_estimators = 100)
forest_fit_10 = forest_10.fit(train_X, train_y)

In [107]:
test_y_predicted = forest_10.predict(test_X)

In [108]:
# 績效
accuracy = metrics.accuracy_score(test_y, test_y_predicted)
print(accuracy)

print(forest.score(test_X, test_y))

0.9933608886026762
0.9933652694086131


In [109]:
#把玉山的 test data 放入 model 做預測
test_y_predicted_jade_test_10 = forest_10.predict(jade_test)

print(test_y_predicted_jade_test_10)
test_y_predicted_jade_test_10.shape

[0 0 0 ... 0 0 0]


(421665,)

### 輸出成 excel 

#### 1. 整理成 dataframe

In [22]:
txkey = jade_test["txkey"] 
# fraud_ind = test_y_predicted_jade_test
fraud_ind = test_y_predicted_jade_test_10

dict = {"txkey": txkey,  
        "fraud_ind": fraud_ind
       }

test_df = pd.DataFrame(data = dict, columns = ["txkey", "fraud_ind"])

test_df.head(6)

,txkey,fraud_ind
0,592489,0
1,592452,0
2,590212,0
3,590209,0
4,592488,0
5,592460,0


#### 2. 輸出至 excel （這個檔案的格式是可以直接上傳到官網的）

In [23]:
import os
print("檔案會存在該路徑下：", os.getcwd())
export_csv = test_df.to_csv (r'submission_'+time.strftime("%Y-%m-%d %H%M%S", time.localtime())+'.csv', 
                             index = None, header = True) 
#Don't forget to add '.csv' at the end of the path

檔案會存在該路徑下： /Users/leosu/Desktop/Jade Mountain
